In [1]:
# =========================================
# PREPROCESSING RECON (Notebook Terpisah)
# Tujuan: kumpulkan semua info yang dibutuhkan pipeline P0–P7
# Output: metadata_preproc.json + beberapa CSV ringkasan
# =========================================

import os, json, re, math
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 220)

# ============ (A) CONFIG ============
DATA_PATH = "/kaggle/input/cyber-attack/Cyber Attack/DataTrain_Preliminary.csv"   # <-- GANTI
TARGET_COL = "type_of_attack"                  # <-- GANTI kalau target beda
RANDOM_SEED = 42

# Token yang sering muncul di dataset log
MISSING_STR_TOKENS = {"nan", "none", "null", ""}     # string missing
PLACEHOLDER_TOKENS = {"*"}                           # missing implisit
SENTINEL_NUMBERS = {99999}                           # sentinel numeric (bisa tambah)

# Batas heuristik untuk deteksi tipe kolom
MAX_UNIQUE_TO_BE_CATEGORICAL = 50
NEAR_CONSTANT_TOP_RATIO = 0.995
HIGH_CARDINALITY_RATIO = 0.5   # unique/rows > 0.5 curiga ID-like

# ============ (B) LOAD DATA ============
assert os.path.exists(DATA_PATH), f"File tidak ditemukan: {DATA_PATH}"
df = pd.read_csv(DATA_PATH, sep = ";")

print("Shape:", df.shape)
print("Columns:", len(df.columns))
print("Target col exists?", TARGET_COL in df.columns)
display(df.head(3))


# ============ (C) BASIC SCHEMA SUMMARY ============
schema = pd.DataFrame({
    "col": df.columns,
    "dtype": [str(df[c].dtype) for c in df.columns],
    "na_count": [int(df[c].isna().sum()) for c in df.columns],
    "na_rate": [float(df[c].isna().mean()) for c in df.columns],
    "n_unique": [int(df[c].nunique(dropna=True)) for c in df.columns],
})
display(schema.sort_values(["dtype","na_rate"], ascending=[True, False]).head(30))


# ============ (D) TOKEN SCAN FOR OBJECT COLUMNS ============
def normalize_str_series(s: pd.Series) -> pd.Series:
    # safe normalize only for non-null
    return s.astype(str).str.strip().str.lower()

def scan_object_tokens(df: pd.DataFrame, topk: int = 10):
    rows = []
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    for c in obj_cols:
        s = normalize_str_series(df[c])
        # hitung token spesifik
        counts = s.value_counts(dropna=False)
        total = len(s)
        # proporsi placeholder/missing-string
        miss_str_rate = float(sum(counts.get(t, 0) for t in MISSING_STR_TOKENS) / total)
        placeholder_rate = float(sum(counts.get(t, 0) for t in PLACEHOLDER_TOKENS) / total)
        nan_literal_rate = float(counts.get("nan", 0) / total)
        star_rate = float(counts.get("*", 0) / total)
        top_tokens = counts.head(topk).to_dict()
        rows.append({
            "col": c,
            "miss_str_rate": miss_str_rate,
            "placeholder_rate": placeholder_rate,
            "nan_literal_rate": nan_literal_rate,
            "star_rate": star_rate,
            "n_unique": int(df[c].nunique(dropna=True)),
            "top_tokens": top_tokens
        })
    return pd.DataFrame(rows).sort_values(["placeholder_rate","nan_literal_rate","miss_str_rate"], ascending=False)

obj_token_report = scan_object_tokens(df, topk=8)
display(obj_token_report.head(25))

# Simpan token report ringkas (top tokens tidak nyaman di csv, jadi json nanti)
obj_token_report.drop(columns=["top_tokens"]).to_csv("object_token_report.csv", index=False)
print("Saved: object_token_report.csv")


# ============ (E) SENTINEL & MIXED-TYPE SCAN (NUMERIC-LIKE OBJECT) ============
numeric_regex = re.compile(r"^\s*[-+]?\d+(\.\d+)?\s*$")

def numeric_likeness_rate(s: pd.Series) -> float:
    s2 = normalize_str_series(s)
    # treat tokens like '*' or 'nan' as non-numeric
    is_num = s2.apply(lambda x: bool(numeric_regex.match(x)))
    return float(is_num.mean())

def scan_numeric_like_objects(df: pd.DataFrame):
    rows = []
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    for c in obj_cols:
        rate = numeric_likeness_rate(df[c])
        rows.append({"col": c, "numeric_likeness_rate": rate, "n_unique": int(df[c].nunique(dropna=True))})
    rep = pd.DataFrame(rows).sort_values("numeric_likeness_rate", ascending=False)
    return rep

numlike_report = scan_numeric_like_objects(df)
display(numlike_report.head(30))
numlike_report.to_csv("object_numeric_likeness.csv", index=False)
print("Saved: object_numeric_likeness.csv")


# ============ (F) NUMERIC DISTRIBUTION FLAGS: ZERO-INFLATION, SENTINEL 99999, EXTREMES ============
def coerce_numeric_preview(series: pd.Series) -> pd.Series:
    # Preview casting: convert common tokens to NaN first (string case)
    if series.dtype == "object":
        s = normalize_str_series(series)
        s = s.replace(list(MISSING_STR_TOKENS | PLACEHOLDER_TOKENS), np.nan)
        return pd.to_numeric(s, errors="coerce")
    else:
        return pd.to_numeric(series, errors="coerce")

def numeric_health_report(df: pd.DataFrame, exclude_cols=None):
    exclude_cols = set(exclude_cols or [])
    rows = []
    for c in df.columns:
        if c in exclude_cols:
            continue
        s_num = coerce_numeric_preview(df[c])
        # skip completely non-numeric
        non_nan = s_num.notna().sum()
        if non_nan == 0:
            continue
        total = len(s_num)
        zero_rate = float((s_num.fillna(0) == 0).mean())
        # sentinel rate for 99999 (and others)
        sentinel_rates = {str(v): float((s_num == v).mean()) for v in SENTINEL_NUMBERS}
        # quantiles (robust)
        q = s_num.quantile([0.5, 0.95, 0.99], interpolation="linear")
        rows.append({
            "col": c,
            "non_nan_rate_after_preview_cast": float(non_nan / total),
            "na_rate_after_preview_cast": float(1 - non_nan / total),
            "zero_rate": zero_rate,
            **{f"sentinel_{k}_rate": v for k,v in sentinel_rates.items()},
            "median": float(q.loc[0.5]) if not math.isnan(q.loc[0.5]) else np.nan,
            "q95": float(q.loc[0.95]) if not math.isnan(q.loc[0.95]) else np.nan,
            "q99": float(q.loc[0.99]) if not math.isnan(q.loc[0.99]) else np.nan,
            "max": float(s_num.max()) if non_nan else np.nan,
        })
    rep = pd.DataFrame(rows)
    return rep.sort_values(["na_rate_after_preview_cast","zero_rate"], ascending=False)

exclude = [TARGET_COL] if TARGET_COL in df.columns else []
num_health = numeric_health_report(df, exclude_cols=exclude)
display(num_health.head(40))
num_health.to_csv("numeric_health_report.csv", index=False)
print("Saved: numeric_health_report.csv")


# ============ (G) CATEGORICAL CANDIDATES & CARDINALITY/DOMINANCE ============
def categorical_report(df: pd.DataFrame, target_col: str):
    rows = []
    n = len(df)
    for c in df.columns:
        if c == target_col:
            continue
        s = df[c]
        # treat object as categorical candidate, also low-unique numerics
        nuniq = int(s.nunique(dropna=True))
        if s.dtype == "object" or nuniq <= MAX_UNIQUE_TO_BE_CATEGORICAL:
            vc = s.value_counts(dropna=False)
            top = vc.iloc[0]
            top_ratio = float(top / n)
            rows.append({
                "col": c,
                "dtype": str(s.dtype),
                "n_unique": nuniq,
                "top_ratio": top_ratio,
                "top_value": str(vc.index[0]),
                "is_near_constant": top_ratio >= NEAR_CONSTANT_TOP_RATIO,
                "unique_ratio": float(nuniq / n),
                "is_high_cardinality": float(nuniq / n) >= HIGH_CARDINALITY_RATIO
            })
    rep = pd.DataFrame(rows).sort_values(["is_near_constant","n_unique"], ascending=[False, False])
    return rep

cat_rep = categorical_report(df, TARGET_COL)
display(cat_rep.head(50))
cat_rep.to_csv("categorical_report.csv", index=False)
print("Saved: categorical_report.csv")


# ============ (H) DUPLICATE CHECK & FLOW SIGNATURE SUGGESTION ============
# Full-row duplicates
dup_mask = df.duplicated(keep=False)
dup_rate = float(dup_mask.mean())
print(f"Full-row duplicate rate: {dup_rate:.4f} ({dup_mask.sum()} rows)")

# Heuristic flow signature: pilih subset kolom yang tidak near-constant & bukan target
# (Anda bisa override manual setelah lihat report)
candidate_cols = [c for c in df.columns if c != TARGET_COL]
# buang kolom near-constant (dari cat report)
near_constant_cols = set(cat_rep.loc[cat_rep["is_near_constant"] == True, "col"].tolist())
sig_cols = [c for c in candidate_cols if c not in near_constant_cols]

# jika terlalu banyak kolom, ambil kolom yang "stabil" (low missing after preview cast + bukan high-cardinality)
high_card_cols = set(cat_rep.loc[cat_rep["is_high_cardinality"] == True, "col"].tolist())
sig_cols = [c for c in sig_cols if c not in high_card_cols]

# limit
sig_cols = sig_cols[:15] if len(sig_cols) > 15 else sig_cols
print("Proposed signature cols (heuristic):", sig_cols)

# group_id & group leakage diagnostic
if len(sig_cols) > 0:
    group_id = pd.util.hash_pandas_object(df[sig_cols].astype(str), index=False).astype("uint64")
    n_groups = group_id.nunique()
    print("n_groups:", n_groups, "group_ratio:", n_groups / len(df))
    # how many groups are size>1
    group_sizes = group_id.value_counts()
    multi_rate = float((group_sizes > 1).mean())
    print("fraction of groups with size>1:", multi_rate)
    group_sizes.head(10).to_frame("size").to_csv("group_sizes_head.csv")
    print("Saved: group_sizes_head.csv")


# ============ (I) BUILD METADATA FOR PREPROCESS PIPELINE ============
metadata = {}

metadata["dataset"] = {
    "shape": list(df.shape),
    "target_col": TARGET_COL,
    "columns": df.columns.tolist(),
    "dtypes": {c: str(df[c].dtype) for c in df.columns},
}

metadata["tokens"] = {
    "missing_str_tokens": sorted(list(MISSING_STR_TOKENS)),
    "placeholder_tokens": sorted(list(PLACEHOLDER_TOKENS)),
    "sentinel_numbers": sorted(list(SENTINEL_NUMBERS)),
}

metadata["reports"] = {
    "schema_csv": "schema_summary.csv",
    "object_token_report_csv": "object_token_report.csv",
    "object_numeric_likeness_csv": "object_numeric_likeness.csv",
    "numeric_health_report_csv": "numeric_health_report.csv",
    "categorical_report_csv": "categorical_report.csv",
}

# Save schema
schema.to_csv("schema_summary.csv", index=False)

# Decide preliminary column groups for later steps
# (Anda boleh edit manual setelah lihat report, tapi ini baseline otomatis)
obj_cols = [c for c in df.columns if df[c].dtype == "object" and c != TARGET_COL]
num_cols = [c for c in df.columns if c != TARGET_COL and df[c].dtype != "object"]

# categorical candidates
cat_candidates = cat_rep["col"].tolist()

metadata["column_groups"] = {
    "object_cols": obj_cols,
    "numeric_non_object_cols": num_cols,
    "categorical_candidates": cat_candidates,
    "near_constant_cols": sorted(list(near_constant_cols)),
    "high_cardinality_cols": sorted(list(high_card_cols)),
    "proposed_signature_cols": sig_cols,
}

# Duplicate stats
metadata["duplicates"] = {
    "full_row_duplicate_rate": dup_rate,
    "full_row_duplicate_rows": int(dup_mask.sum())
}

# Persist json metadata
with open("metadata_preproc.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Saved: schema_summary.csv")
print("✅ Saved: metadata_preproc.json")


Shape: (112446, 42)
Columns: 42
Target col exists? True


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
0,0,tcp,private,SH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,176,1,0.01,0.94,0.95,0,0.95,1,0,0,nmap
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,*,0,0,0,0,0,0,0,144,5,1,1,0,0,*,*,0,255,5,0.02,0.08,0.00,0,1.00,1,0,0,neptune
2,0,tcp,http,SF,285,3623,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,15,0,0,0,0,1,0,*,115,228,1.00,0.00,0.01,*,0.00,0,0,0,normal


,col,dtype,na_count,na_rate,n_unique
35,dst_host_same_src_port_rate,float64,438,0.003895,102
37,dst_host_serror_rate,float64,428,0.003806,102
33,dst_host_same_srv_rate,float64,328,0.002917,102
34,dst_host_diff_srv_rate,float64,243,0.002161,102
2,service,object,1020,0.009071,70
3,flag,object,853,0.007586,11
1,protocol_type,object,668,0.005941,3
0,duration,object,0,0.000000,2665
4,src_bytes,object,0,0.000000,3201
5,dst_bytes,object,0,0.000000,8843


,col,miss_str_rate,placeholder_rate,nan_literal_rate,star_rate,n_unique,top_tokens
28,same_srv_rate,0.0,0.374855,0.0,0.374855,4,"{'1': 67573, '*': 42151, '0': 2443, '99999': 279}"
29,diff_srv_rate,0.0,0.372534,0.0,0.372534,4,"{'0': 67147, '*': 41890, '1': 3057, '99999': 352}"
33,dst_host_srv_diff_host_rate,0.0,0.301994,0.0,0.301994,4,"{'0': 77261, '*': 33958, '1': 626, '99999': 601}"
30,srv_diff_host_rate,0.0,0.163447,0.0,0.163447,3,"{'0': 86738, '*': 18379, '1': 7329}"
35,dst_host_rerror_rate,0.0,0.100937,0.0,0.100937,4,"{'0': 91828, '*': 11350, '1': 9191, '99999': 77}"
36,dst_host_srv_rerror_rate,0.0,0.052176,0.0,0.052176,3,"{'0': 94734, '1': 11845, '*': 5867}"
34,dst_host_srv_serror_rate,0.0,0.052043,0.0,0.052043,3,"{'0': 75749, '1': 30845, '*': 5852}"
24,serror_rate,0.0,0.041033,0.0,0.041033,4,"{'0': 76821, '1': 30908, '*': 4614, '99999': 103}"
26,rerror_rate,0.0,0.030219,0.0,0.030219,4,"{'0': 97130, '1': 11397, '*': 3398, '99999': 521}"
25,srv_serror_rate,0.0,0.022366,0.0,0.022366,4,"{'0': 78336, '1': 31234, '*': 2515, '99999': 361}"


Saved: object_token_report.csv


...

Saved: object_numeric_likeness.csv


,col,non_nan_rate_after_preview_cast,na_rate_after_preview_cast,zero_rate,sentinel_99999_rate,median,q95,q99,max
25,same_srv_rate,0.625145,0.374855,0.396581,0.002481,1.00,1.0,1.00,9.999900e+04
26,diff_srv_rate,0.627466,0.372534,0.969683,0.003130,0.00,0.0,1.00,9.999900e+04
33,dst_host_srv_diff_host_rate,0.698006,0.301994,0.989088,0.005345,0.00,0.0,1.00,9.999900e+04
27,srv_diff_host_rate,0.836553,0.163447,0.934822,0.000000,0.00,1.0,1.00,1.000000e+00
36,dst_host_rerror_rate,0.899063,0.100937,0.917578,0.000685,0.00,1.0,1.00,9.999900e+04
37,dst_host_srv_rerror_rate,0.947824,0.052176,0.894661,0.000000,0.00,1.0,1.00,1.000000e+00
35,dst_host_srv_serror_rate,0.947957,0.052043,0.725691,0.000000,0.00,1.0,1.00,1.000000e+00
21,serror_rate,0.958967,0.041033,0.724214,0.000916,0.00,1.0,1.00,9.999900e+04
23,rerror_rate,0.969781,0.030219,0.894011,0.004633,0.00,1.0,1.00,9.999900e+04
22,srv_serror_rate,0.977634,0.022366,0.719021,0.003210,0.00,1.0,1.00,9.999900e+04


Saved: numeric_health_report.csv


,col,dtype,n_unique,top_ratio,top_value,is_near_constant,unique_ratio,is_high_cardinality
16,num_file_creations,object,33,0.995794,0,True,0.000293,False
8,urgent,object,6,0.996683,0,True,0.000053,False
14,su_attempted,object,5,0.996923,0,True,0.000044,False
17,num_shells,object,5,0.995331,0,True,0.000044,False
6,land,object,4,0.996167,0,True,0.000036,False
13,root_shell,object,4,0.996114,0,True,0.000036,False
20,is_host_login,object,3,0.999315,0,True,0.000027,False
19,num_outbound_cmds,object,2,0.998346,0,True,0.000018,False
5,dst_bytes,object,8843,0.534817,0,False,0.078642,False
4,src_bytes,object,3201,0.391566,0,False,0.028467,False


Saved: categorical_report.csv
Full-row duplicate rate: 0.0198 (2230 rows)
Proposed signature cols (heuristic): ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'num_root', 'num_access_files', 'is_guest_login', 'count']
n_groups: 66422 group_ratio: 0.590701314408694
fraction of groups with size>1: 0.1378760049381229
Saved: group_sizes_head.csv
✅ Saved: schema_summary.csv
✅ Saved: metadata_preproc.json


In [2]:
DATA_PATH = "/kaggle/input/cyber-attack/Cyber Attack/DataTrain_Preliminary.csv"  # <-- GANTI sesuai dataset Anda

In [3]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import GroupShuffleSplit

# --- Load metadata ---
META_PATH = "/kaggle/working/metadata_preproc.json"
with open(META_PATH, "r") as f:
    meta = json.load(f)

TARGET_COL = meta["dataset"]["target_col"]
SIG_COLS = meta["column_groups"]["proposed_signature_cols"]

# --- Load raw data ---
df = pd.read_csv(DATA_PATH, sep =";")
assert TARGET_COL in df.columns, f"Target column '{TARGET_COL}' tidak ada!"

# --- Output dir (ini yang akan muncul di panel kanan Kaggle) ---
OUT_DIR = Path("./artifacts_p0")
OUT_DIR.mkdir(exist_ok=True, parents=True)

print("Raw shape:", df.shape)
print("TARGET_COL:", TARGET_COL)
print("Signature cols:", SIG_COLS)

# --- Full-row dedup (ON) ---
dup_mask = df.duplicated(keep="first")
n_dup = int(dup_mask.sum())
df = df.loc[~dup_mask].reset_index(drop=True)
print("Dropped duplicates:", n_dup)
print("After dedup:", df.shape)

# --- Build group_id from signature ---
missing_sig = [c for c in SIG_COLS if c not in df.columns]
if len(missing_sig) > 0:
    raise ValueError(f"Signature cols missing in dataset: {missing_sig}")

sig_df = df[SIG_COLS].astype(str)
df["__group_id__"] = pd.util.hash_pandas_object(sig_df, index=False).astype("uint64")

# --- Group-aware split ---
VAL_SIZE = 0.20
RANDOM_SEED = 42

gss = GroupShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=RANDOM_SEED)
X_dummy = np.zeros((len(df), 1))
y = df[TARGET_COL].astype(str).values
groups = df["__group_id__"].values

train_idx, val_idx = next(gss.split(X_dummy, y=y, groups=groups))
train_idx = np.array(train_idx)
val_idx = np.array(val_idx)

# sanity: no group overlap
overlap = len(set(df.loc[train_idx, "__group_id__"]).intersection(set(df.loc[val_idx, "__group_id__"])))
print("Train rows:", len(train_idx), "| Val rows:", len(val_idx))
print("Group overlap (must be 0):", overlap)
assert overlap == 0, "Group leakage detected! (overlap != 0)"

# --- Save artifacts ---
df.to_csv(OUT_DIR / "df_p0.csv", index=False)
pd.DataFrame({"idx": train_idx}).to_csv(OUT_DIR / "train_idx.csv", index=False)
pd.DataFrame({"idx": val_idx}).to_csv(OUT_DIR / "val_idx.csv", index=False)

p0_artifacts = {
    "stage": "P0",
    "target_col": TARGET_COL,
    "signature_cols": SIG_COLS,
    "val_size": VAL_SIZE,
    "random_seed": RANDOM_SEED,
    "n_rows_after_dedup": int(len(df)),
    "n_groups": int(df["__group_id__"].nunique()),
    "dropped_full_row_duplicates": n_dup
}
with open(OUT_DIR / "p0_artifacts.json", "w") as f:
    json.dump(p0_artifacts, f, indent=2)

print("✅ Saved artifacts to:", str(OUT_DIR))
print(list(OUT_DIR.glob("*")))

Raw shape: (112446, 42)
TARGET_COL: type_of_attack
Signature cols: ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'num_root', 'num_access_files', 'is_guest_login', 'count']
Dropped duplicates: 1407
After dedup: (111039, 42)
Train rows: 90129 | Val rows: 20910
Group overlap (must be 0): 0
✅ Saved artifacts to: artifacts_p0
[PosixPath('artifacts_p0/train_idx.csv'), PosixPath('artifacts_p0/p0_artifacts.json'), PosixPath('artifacts_p0/df_p0.csv'), PosixPath('artifacts_p0/val_idx.csv')]


# P1 - Token Normalization (Object-heavy → token bersih)

In [4]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

P0_DIR = Path("./artifacts_p0")
P1_DIR = Path("./artifacts_p1")
P1_DIR.mkdir(exist_ok=True, parents=True)

# --- must exist ---
assert (P0_DIR / "df_p0.csv").exists(), "df_p0.csv tidak ditemukan. Pastikan P0 sudah dijalankan & tersimpan."
assert (P0_DIR / "p0_artifacts.json").exists(), "p0_artifacts.json tidak ditemukan. Pastikan P0 sudah dijalankan & tersimpan."

df = pd.read_csv(P0_DIR / "df_p0.csv")
with open(P0_DIR / "p0_artifacts.json", "r") as f:
    p0 = json.load(f)

TARGET_COL = p0["target_col"]
print("Loaded df_p0:", df.shape, "| TARGET_COL:", TARGET_COL)

# token rules
MISSING_STR_TOKENS = {"nan", "none", "null", ""}   # string missing tokens
PLACEHOLDER_TOKENS = {"*"}                         # placeholder / missing implisit

Loaded df_p0: (111039, 43) | TARGET_COL: type_of_attack


In [5]:
obj_cols = [c for c in df.columns if df[c].dtype == "object" and c != TARGET_COL]
print("Object cols to clean:", len(obj_cols))

def normalize_series_tokens(s: pd.Series):
    """
    - strip + lower
    - replace missing tokens & placeholder tokens to NaN
    Returns: cleaned_series, stats
    """
    s_str = s.astype("string")                      # preserves <NA>
    s_norm = s_str.str.strip().str.lower()

    # counts BEFORE replace
    vc = s_norm.value_counts(dropna=False)
    n = len(s_norm)

    miss_cnt = int(sum(vc.get(t, 0) for t in MISSING_STR_TOKENS))
    ph_cnt   = int(sum(vc.get(t, 0) for t in PLACEHOLDER_TOKENS))

    # replace
    s_norm = s_norm.replace(list(MISSING_STR_TOKENS | PLACEHOLDER_TOKENS), pd.NA)

    # output as object with np.nan
    out = s_norm.astype("object").where(lambda x: x.notna(), np.nan)

    stats = {
        "n_rows": n,
        "missing_token_replaced": miss_cnt,
        "placeholder_replaced": ph_cnt,
        "replaced_total": miss_cnt + ph_cnt,
        "replaced_rate": float((miss_cnt + ph_cnt) / n)
    }
    return out, stats

report_rows = []
for c in obj_cols:
    before_na = float(df[c].isna().mean())
    cleaned, stats = normalize_series_tokens(df[c])
    df[c] = cleaned
    after_na = float(df[c].isna().mean())

    report_rows.append({
        "col": c,
        "na_rate_before": before_na,
        "na_rate_after": after_na,
        "delta_na": after_na - before_na,
        **stats
    })

p1_report = pd.DataFrame(report_rows).sort_values(["replaced_rate", "delta_na"], ascending=False)
display(p1_report.head(30))

Object cols to clean: 37


,col,na_rate_before,na_rate_after,delta_na,n_rows,missing_token_replaced,placeholder_replaced,replaced_total,replaced_rate
28,same_srv_rate,0.0,0.376778,0.376778,111039,0,41837,41837,0.376778
29,diff_srv_rate,0.0,0.374355,0.374355,111039,0,41568,41568,0.374355
33,dst_host_srv_diff_host_rate,0.0,0.301984,0.301984,111039,0,33532,33532,0.301984
30,srv_diff_host_rate,0.0,0.164951,0.164951,111039,0,18316,18316,0.164951
35,dst_host_rerror_rate,0.0,0.096164,0.096164,111039,0,10678,10678,0.096164
34,dst_host_srv_serror_rate,0.0,0.052423,0.052423,111039,0,5821,5821,0.052423
36,dst_host_srv_rerror_rate,0.0,0.049235,0.049235,111039,0,5467,5467,0.049235
24,serror_rate,0.0,0.038374,0.038374,111039,0,4261,4261,0.038374
26,rerror_rate,0.0,0.027252,0.027252,111039,0,3026,3026,0.027252
25,srv_serror_rate,0.0,0.022560,0.022560,111039,0,2505,2505,0.022560


In [6]:
tokens_check = MISSING_STR_TOKENS | PLACEHOLDER_TOKENS

bad = []
for c in obj_cols:
    s = df[c].dropna().astype(str).str.strip().str.lower()
    leftover = int(s.isin(tokens_check).sum())
    if leftover > 0:
        bad.append((c, leftover))

print("Cols with leftover tokens:", len(bad))
print("Example:", bad[:10])

Cols with leftover tokens: 0
Example: []


In [7]:
df.to_csv(P1_DIR / "df_p1.csv", index=False)
p1_report.to_csv(P1_DIR / "p1_report.csv", index=False)

p1_artifacts = {
    "stage": "P1",
    "target_col": TARGET_COL,
    "missing_str_tokens": sorted(list(MISSING_STR_TOKENS)),
    "placeholder_tokens": sorted(list(PLACEHOLDER_TOKENS)),
    "object_cols_cleaned": obj_cols,
    "n_object_cols_cleaned": int(len(obj_cols)),
    "n_rows": int(len(df))
}

with open(P1_DIR / "p1_artifacts.json", "w") as f:
    json.dump(p1_artifacts, f, indent=2)

print("✅ Saved:", P1_DIR / "df_p1.csv")
print("✅ Saved:", P1_DIR / "p1_report.csv")
print("✅ Saved:", P1_DIR / "p1_artifacts.json")

✅ Saved: artifacts_p1/df_p1.csv
✅ Saved: artifacts_p1/p1_report.csv
✅ Saved: artifacts_p1/p1_artifacts.json


# P2 - Semantic Casting (Cast bertahap + audit)

In [8]:
import json
import numpy as np
import pandas as pd
import re
from pathlib import Path

P1_DIR = Path("./artifacts_p1")
P2_DIR = Path("./artifacts_p2")
P2_DIR.mkdir(exist_ok=True, parents=True)

assert (P1_DIR / "df_p1.csv").exists(), "df_p1.csv tidak ditemukan. Jalankan P1 dulu."
assert (P1_DIR / "p1_artifacts.json").exists(), "p1_artifacts.json tidak ditemukan."

df = pd.read_csv(P1_DIR / "df_p1.csv")
with open(P1_DIR / "p1_artifacts.json", "r") as f:
    p1 = json.load(f)

TARGET_COL = p1["target_col"]
print("Loaded df_p1:", df.shape, "| TARGET_COL:", TARGET_COL)

# Thresholds (konservatif)
NUMERIC_SAFE_THRESHOLD = 0.995   # >=99.5% nilai non-null bisa jadi angka -> numeric-safe
MIXED_MIN_THRESHOLD = 0.50       # 50%–99.5% numeric-like -> mixed (perlu keputusan)
MAX_UNIQUE_TO_BE_CATEGORICAL = 50  # fallback: object low-unique -> categorical candidate

Loaded df_p1: (111039, 43) | TARGET_COL: type_of_attack


In [9]:
numeric_regex = re.compile(r"^\s*[-+]?\d+(\.\d+)?\s*$")

def numeric_likeness(s: pd.Series) -> float:
    """Proporsi nilai non-null yang match angka murni."""
    s2 = s.dropna().astype(str)
    if len(s2) == 0:
        return 0.0
    ok = s2.apply(lambda x: bool(numeric_regex.match(x)))
    return float(ok.mean())

def sample_non_numeric_tokens(s: pd.Series, k=10):
    """Ambil contoh token non-numeric (setelah P1) untuk audit mixed-type."""
    s2 = s.dropna().astype(str)
    if len(s2) == 0:
        return []
    bad = s2[~s2.apply(lambda x: bool(numeric_regex.match(x)))]
    return bad.value_counts().head(k).index.tolist()


In [10]:
obj_cols = [c for c in df.columns if df[c].dtype == "object" and c != TARGET_COL]

rows = []
for c in obj_cols:
    s = df[c]
    like = numeric_likeness(s)
    nuniq = int(s.nunique(dropna=True))
    na_rate = float(s.isna().mean())

    # rule-based grouping
    if like >= NUMERIC_SAFE_THRESHOLD:
        group = "numeric_safe"
    elif like >= MIXED_MIN_THRESHOLD:
        group = "mixed_type"
    else:
        # numeric-likeness rendah -> cenderung categorical
        group = "categorical"

    # low-unique objects are almost always categorical (override)
    if nuniq <= MAX_UNIQUE_TO_BE_CATEGORICAL and like < NUMERIC_SAFE_THRESHOLD:
        group = "categorical"

    rows.append({
        "col": c,
        "na_rate_obj": na_rate,
        "n_unique_obj": nuniq,
        "numeric_likeness": like,
        "group": group,
        "sample_bad_tokens": sample_non_numeric_tokens(s, k=8) if group == "mixed_type" else []
    })

p2_scan = pd.DataFrame(rows).sort_values(["group", "numeric_likeness"], ascending=[True, False])
display(p2_scan.head(40))

# Save scan for review
p2_scan.to_csv(P2_DIR / "p2_object_scan.csv", index=False)
print("✅ Saved:", P2_DIR / "p2_object_scan.csv")


,col,na_rate_obj,n_unique_obj,numeric_likeness,group,sample_bad_tokens
0,protocol_type,0.006016,3,0.0,categorical,[]
1,service,0.009078,70,0.0,categorical,[]
2,flag,0.007673,11,0.0,categorical,[]


✅ Saved: artifacts_p2/p2_object_scan.csv


In [11]:
display(
    p2_scan.groupby("group")["col"]
    .count()
    .rename("n_cols")
    .to_frame()
)
display(p2_scan.sort_values("numeric_likeness", ascending=False).head(20))

,n_cols
group,
categorical,3


,col,na_rate_obj,n_unique_obj,numeric_likeness,group,sample_bad_tokens
0,protocol_type,0.006016,3,0.0,categorical,[]
1,service,0.009078,70,0.0,categorical,[]
2,flag,0.007673,11,0.0,categorical,[]


In [12]:
# --- Casting terbatas hanya untuk numeric-safe ---
numeric_safe_cols = p2_scan.loc[p2_scan["group"] == "numeric_safe", "col"].tolist()
mixed_type_cols   = p2_scan.loc[p2_scan["group"] == "mixed_type", "col"].tolist()
categorical_cols  = p2_scan.loc[p2_scan["group"] == "categorical", "col"].tolist()

print("numeric_safe_cols:", len(numeric_safe_cols))
print("mixed_type_cols:", len(mixed_type_cols))
print("categorical_cols:", len(categorical_cols))

cast_report_rows = []

if len(numeric_safe_cols) == 0:
    print("⚠️ Tidak ada object yang numeric-safe pada threshold saat ini.")
    print("   Ini OK. Kita tidak melakukan casting di P2 untuk object. Mixed-type akan ditangani di P3.")
else:
    for c in numeric_safe_cols:
        s0 = df[c]
        na_before = float(s0.isna().mean())
        nonnull_before = int(s0.notna().sum())

        s_num = pd.to_numeric(s0, errors="coerce")
        na_after = float(s_num.isna().mean())
        nonnull_after = int(s_num.notna().sum())

        new_nan = max(0, nonnull_before - nonnull_after)

        df[c] = s_num

        cast_report_rows.append({
            "col": c,
            "na_before": na_before,
            "na_after": na_after,
            "new_nan_from_cast": int(new_nan),
            "new_nan_rate_from_cast": float(new_nan / len(df))
        })

cast_report = pd.DataFrame(cast_report_rows)

if len(cast_report) > 0:
    cast_report = cast_report.sort_values("new_nan_rate_from_cast", ascending=False)
    display(cast_report.head(30))
    cast_report.to_csv(P2_DIR / "p2_cast_report_numeric_safe.csv", index=False)
    print("✅ Saved:", P2_DIR / "p2_cast_report_numeric_safe.csv")
else:
    # tetap buat file report kosong agar pipeline konsisten
    cast_report.to_csv(P2_DIR / "p2_cast_report_numeric_safe.csv", index=False)
    print("✅ Saved empty:", P2_DIR / "p2_cast_report_numeric_safe.csv")

numeric_safe_cols: 0
mixed_type_cols: 0
categorical_cols: 3
⚠️ Tidak ada object yang numeric-safe pada threshold saat ini.
   Ini OK. Kita tidak melakukan casting di P2 untuk object. Mixed-type akan ditangani di P3.
✅ Saved empty: artifacts_p2/p2_cast_report_numeric_safe.csv


In [13]:
# ============================
# P2 — Cell 5: Guardrail Audit
# ============================

ALERT_NEW_NAN_RATE = 0.001  # 0.1%

# Jika cast_report kosong atau kolom audit tidak ada, skip
if cast_report is None or len(cast_report) == 0 or "new_nan_rate_from_cast" not in cast_report.columns:
    print("⚠️ Skip audit: tidak ada kolom numeric-safe yang dicasting pada P2 (cast_report kosong).")
    alerts = pd.DataFrame()
else:
    alerts = cast_report.loc[cast_report["new_nan_rate_from_cast"] > ALERT_NEW_NAN_RATE].copy()
    print("ALERT columns (new NaN too high):", len(alerts))
    display(alerts.head(20))

    # Optional: contoh token gagal (hanya untuk kolom alert)
    def failed_tokens_after_cast(col, k=10):
        raw = pd.read_csv(P1_DIR / "df_p1.csv", usecols=[col])[col]
        raw_nonnull = raw.dropna().astype(str)
        bad = raw_nonnull[pd.to_numeric(raw_nonnull, errors="coerce").isna()]
        return bad.value_counts().head(k)

    for c in alerts["col"].tolist()[:10]:
        print("\n--- Failed token examples for:", c)
        display(failed_tokens_after_cast(c, k=10))

⚠️ Skip audit: tidak ada kolom numeric-safe yang dicasting pada P2 (cast_report kosong).


In [14]:
# Save df after casting numeric-safe
df.to_csv(P2_DIR / "df_p2.csv", index=False)

p2_groups = {
    "target_col": TARGET_COL,
    "numeric_safe_cols": numeric_safe_cols,
    "mixed_type_cols": mixed_type_cols,
    "categorical_cols": categorical_cols,
    "casted_numeric_cols": numeric_safe_cols,  # casted in P2
    "kept_object_cols": mixed_type_cols + categorical_cols
}

with open(P2_DIR / "p2_column_groups.json", "w") as f:
    json.dump(p2_groups, f, indent=2)

print("✅ Saved:", P2_DIR / "df_p2.csv")
print("✅ Saved:", P2_DIR / "p2_column_groups.json")
print("✅ Saved:", P2_DIR / "p2_object_scan.csv")


✅ Saved: artifacts_p2/df_p2.csv
✅ Saved: artifacts_p2/p2_column_groups.json
✅ Saved: artifacts_p2/p2_object_scan.csv


# P3 - Value–State Decomposition

In [15]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

P2_DIR = Path("./artifacts_p2")
P1_DIR = Path("./artifacts_p1")
P3_DIR = Path("./artifacts_p3")
P3_DIR.mkdir(exist_ok=True, parents=True)

assert (P2_DIR / "df_p2.csv").exists(), "df_p2.csv tidak ditemukan. Pastikan P2 sudah tersimpan."
assert (P1_DIR / "p1_report.csv").exists(), "p1_report.csv tidak ditemukan. Pastikan P1 sudah tersimpan."
assert (P1_DIR / "p1_artifacts.json").exists(), "p1_artifacts.json tidak ditemukan."

df = pd.read_csv(P2_DIR / "df_p2.csv")

with open(P1_DIR / "p1_artifacts.json", "r") as f:
    p1 = json.load(f)
TARGET_COL = p1["target_col"]

p1_report = pd.read_csv(P1_DIR / "p1_report.csv")

print("Loaded df_p2:", df.shape, "| TARGET_COL:", TARGET_COL)
display(p1_report.sort_values("delta_na", ascending=False).head(10))

Loaded df_p2: (111039, 43) | TARGET_COL: type_of_attack


,col,na_rate_before,na_rate_after,delta_na,n_rows,missing_token_replaced,placeholder_replaced,replaced_total,replaced_rate
0,same_srv_rate,0.0,0.376778,0.376778,111039,0,41837,41837,0.376778
1,diff_srv_rate,0.0,0.374355,0.374355,111039,0,41568,41568,0.374355
2,dst_host_srv_diff_host_rate,0.0,0.301984,0.301984,111039,0,33532,33532,0.301984
3,srv_diff_host_rate,0.0,0.164951,0.164951,111039,0,18316,18316,0.164951
4,dst_host_rerror_rate,0.0,0.096164,0.096164,111039,0,10678,10678,0.096164
5,dst_host_srv_serror_rate,0.0,0.052423,0.052423,111039,0,5821,5821,0.052423
6,dst_host_srv_rerror_rate,0.0,0.049235,0.049235,111039,0,5467,5467,0.049235
7,serror_rate,0.0,0.038374,0.038374,111039,0,4261,4261,0.038374
8,rerror_rate,0.0,0.027252,0.027252,111039,0,3026,3026,0.027252
9,srv_serror_rate,0.0,0.022560,0.022560,111039,0,2505,2505,0.022560


In [16]:
# ============================
# P3 — Cell 2 (FIXED)
# ============================

IMPLICIT_MISSING_DELTA_NA_THRESHOLD = 0.001  # 0.1%

# --- auto detect nama kolom kolom-name di p1_report ---
possible_name_cols = ["column", "col"]
name_col = None
for cand in possible_name_cols:
    if cand in p1_report.columns:
        name_col = cand
        break

assert name_col is not None, f"Tidak menemukan kolom nama di p1_report. Kolom tersedia: {list(p1_report.columns)}"
assert "delta_na" in p1_report.columns, f"Tidak ada kolom 'delta_na' di p1_report. Kolom tersedia: {list(p1_report.columns)}"

implicit_missing_cols = (
    p1_report.loc[p1_report["delta_na"] >= IMPLICIT_MISSING_DELTA_NA_THRESHOLD, name_col]
    .astype(str)
    .tolist()
)

# pastikan kolomnya ada di df
implicit_missing_cols = [c for c in implicit_missing_cols if c in df.columns and c != TARGET_COL]

print("p1_report name column used:", name_col)
print("Implicit-missing cols (from P1 delta_na):", len(implicit_missing_cols))
print("Example:", implicit_missing_cols[:15])

p1_report name column used: col
Implicit-missing cols (from P1 delta_na): 33
Example: ['same_srv_rate', 'diff_srv_rate', 'dst_host_srv_diff_host_rate', 'srv_diff_host_rate', 'dst_host_rerror_rate', 'dst_host_srv_serror_rate', 'dst_host_srv_rerror_rate', 'serror_rate', 'rerror_rate', 'srv_serror_rate', 'srv_rerror_rate', 'hot', 'src_bytes', 'dst_bytes', 'num_access_files']


In [17]:
SENTINEL_VALUE = 99999

numeric_cols = [c for c in df.columns if c != TARGET_COL and df[c].dtype != "object"]
sentinel_cols = []

for c in numeric_cols:
    s = df[c]
    # hitung rate sentinel (NaN safe)
    rate = float((s == SENTINEL_VALUE).mean()) if len(s) else 0.0
    if rate > 0:
        sentinel_cols.append((c, rate))

sentinel_cols = sorted(sentinel_cols, key=lambda x: x[1], reverse=True)
sentinel_cols_only = [c for c, _ in sentinel_cols]

print("Numeric cols:", len(numeric_cols))
print("Sentinel(99999) cols:", len(sentinel_cols_only))
print("Top sentinel cols:", sentinel_cols[:10])

Numeric cols: 39
Sentinel(99999) cols: 28
Top sentinel cols: [('count', 0.007321751816929187), ('srv_count', 0.0057907582020731456), ('dst_host_srv_diff_host_rate', 0.005412512720755771), ('dst_host_same_srv_rate', 0.0051873666009239995), ('rerror_rate', 0.004683039292500833), ('duration', 0.003269121659957312), ('srv_serror_rate', 0.0032511099703707706), ('diff_srv_rate', 0.003170057367231333), ('same_srv_rate', 0.0024585956285629374), ('dst_host_count', 0.0024495897837696666)]


In [18]:
# Control: jika Anda takut kolom flag terlalu banyak, batasi maksimum
MAX_FLAG_COLS = 999  # set kecil jika perlu, mis. 50

flag_cols_created = []
value_clean_cols = []

# --- (A) implicit missing flags ---
for c in implicit_missing_cols[:MAX_FLAG_COLS]:
    flag_name = f"is_missing_implisit__{c}"
    df[flag_name] = df[c].isna().astype("int8")
    flag_cols_created.append(flag_name)
    # value_clean: kita biarkan df[c] apa adanya (NaN sudah benar)
    # (nanti transform/impute di P4)
    value_clean_cols.append(c)

# --- (B) sentinel 99999 flags + replace to NaN ---
for c in sentinel_cols_only[:MAX_FLAG_COLS]:
    flag_name = f"is_sentinel_99999__{c}"
    df[flag_name] = (df[c] == SENTINEL_VALUE).astype("int8")
    flag_cols_created.append(flag_name)

    # Replace sentinel -> NaN (value_clean)
    df.loc[df[c] == SENTINEL_VALUE, c] = np.nan
    if c not in value_clean_cols:
        value_clean_cols.append(c)

# --- (C) string 'nan' flags for remaining object cols (guard) ---
obj_cols = [c for c in df.columns if df[c].dtype == "object" and c != TARGET_COL]
for c in obj_cols[:MAX_FLAG_COLS]:
    # only if any literal 'nan' still exists (should be rare after P1)
    s = df[c].dropna().astype(str).str.strip().str.lower()
    if (s == "nan").any():
        flag_name = f"is_str_nan__{c}"
        df[flag_name] = df[c].astype("string").str.strip().str.lower().eq("nan").fillna(False).astype("int8")
        flag_cols_created.append(flag_name)

print("Flags created:", len(flag_cols_created))
print("Example flags:", flag_cols_created[:20])

Flags created: 61
Example flags: ['is_missing_implisit__same_srv_rate', 'is_missing_implisit__diff_srv_rate', 'is_missing_implisit__dst_host_srv_diff_host_rate', 'is_missing_implisit__srv_diff_host_rate', 'is_missing_implisit__dst_host_rerror_rate', 'is_missing_implisit__dst_host_srv_serror_rate', 'is_missing_implisit__dst_host_srv_rerror_rate', 'is_missing_implisit__serror_rate', 'is_missing_implisit__rerror_rate', 'is_missing_implisit__srv_serror_rate', 'is_missing_implisit__srv_rerror_rate', 'is_missing_implisit__hot', 'is_missing_implisit__src_bytes', 'is_missing_implisit__dst_bytes', 'is_missing_implisit__num_access_files', 'is_missing_implisit__dst_host_count', 'is_missing_implisit__duration', 'is_missing_implisit__dst_host_srv_count', 'is_missing_implisit__wrong_fragment', 'is_missing_implisit__num_failed_logins']


In [19]:
MAKE_ROW_ANOMALY_SCORE = True

if MAKE_ROW_ANOMALY_SCORE and len(flag_cols_created) > 0:
    df["row_anomaly_score"] = df[flag_cols_created].sum(axis=1).astype("int16")
    print("row_anomaly_score created.")
    print(df["row_anomaly_score"].describe())
else:
    print("row_anomaly_score skipped (no flags or disabled).")

row_anomaly_score created.
count    111039.000000
mean          1.660777
std           1.086065
min           0.000000
25%           1.000000
50%           2.000000
75%           2.000000
max          11.000000
Name: row_anomaly_score, dtype: float64


In [20]:
p3_flag_report = []

for fcol in flag_cols_created:
    p3_flag_report.append({
        "flag_col": fcol,
        "rate": float(df[fcol].mean()),
        "count": int(df[fcol].sum())
    })

p3_flag_report = pd.DataFrame(p3_flag_report).sort_values("rate", ascending=False)
display(p3_flag_report.head(30))

p3_flag_report.to_csv(P3_DIR / "p3_flags_report.csv", index=False)
print("✅ Saved:", P3_DIR / "p3_flags_report.csv")

,flag_col,rate,count
0,is_missing_implisit__same_srv_rate,0.376778,41837
1,is_missing_implisit__diff_srv_rate,0.374355,41568
2,is_missing_implisit__dst_host_srv_diff_host_rate,0.301984,33532
3,is_missing_implisit__srv_diff_host_rate,0.164951,18316
4,is_missing_implisit__dst_host_rerror_rate,0.096164,10678
5,is_missing_implisit__dst_host_srv_serror_rate,0.052423,5821
6,is_missing_implisit__dst_host_srv_rerror_rate,0.049235,5467
7,is_missing_implisit__serror_rate,0.038374,4261
8,is_missing_implisit__rerror_rate,0.027252,3026
9,is_missing_implisit__srv_serror_rate,0.022560,2505


✅ Saved: artifacts_p3/p3_flags_report.csv


In [21]:
df.to_csv(P3_DIR / "df_p3.csv", index=False)

p3_artifacts = {
    "stage": "P3",
    "target_col": TARGET_COL,
    "implicit_missing_delta_na_threshold": IMPLICIT_MISSING_DELTA_NA_THRESHOLD,
    "implicit_missing_cols": implicit_missing_cols,
    "sentinel_value": SENTINEL_VALUE,
    "sentinel_cols_sorted": sentinel_cols,  # list of (col, rate)
    "flag_cols_created": flag_cols_created,
    "made_row_anomaly_score": MAKE_ROW_ANOMALY_SCORE,
}

with open(P3_DIR / "p3_artifacts.json", "w") as f:
    json.dump(p3_artifacts, f, indent=2)

print("✅ Saved:", P3_DIR / "df_p3.csv")
print("✅ Saved:", P3_DIR / "p3_artifacts.json")

✅ Saved: artifacts_p3/df_p3.csv
✅ Saved: artifacts_p3/p3_artifacts.json


# P4 - Missingness & Imputation

In [22]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

P3_DIR = Path("./artifacts_p3")
P0_DIR = Path("./artifacts_p0")
P4_DIR = Path("./artifacts_p4")
P4_DIR.mkdir(exist_ok=True, parents=True)

assert (P3_DIR / "df_p3.csv").exists(), "df_p3.csv tidak ditemukan. Jalankan P3 dulu."
assert (P0_DIR / "train_idx.csv").exists(), "train_idx.csv tidak ditemukan. Jalankan P0 dulu."
assert (P3_DIR / "p3_artifacts.json").exists(), "p3_artifacts.json tidak ditemukan."

df = pd.read_csv(P3_DIR / "df_p3.csv")
train_idx = pd.read_csv(P0_DIR / "train_idx.csv")["idx"].values

with open(P3_DIR / "p3_artifacts.json", "r") as f:
    p3 = json.load(f)

TARGET_COL = p3["target_col"]

print("Loaded df_p3:", df.shape)
print("Train rows:", len(train_idx))

Loaded df_p3: (111039, 105)
Train rows: 90129


In [23]:
# Identifikasi tipe kolom
flag_cols = [c for c in df.columns if c.startswith("is_")]
numeric_cols = [
    c for c in df.columns
    if c not in flag_cols
    and c != TARGET_COL
    and df[c].dtype != "object"
]

print("Numeric cols:", len(numeric_cols))
print("Flag cols:", len(flag_cols))

Numeric cols: 38
Flag cols: 63


In [24]:
imputer_stats = []
imputers = {}

train_df = df.loc[train_idx]

for c in numeric_cols:
    s = train_df[c]

    # skip jika tidak ada missing
    miss_rate = float(s.isna().mean())
    if miss_rate == 0:
        continue

    # robust statistics
    median = float(s.median())
    q50 = float(s.quantile(0.50))
    q95 = float(s.quantile(0.95))

    imputers[c] = {
        "strategy": "median",
        "median": median,
        "q50": q50,
        "q95": q95,
        "missing_rate_train": miss_rate
    }

    imputer_stats.append({
        "col": c,
        "missing_rate_train": miss_rate,
        "impute_strategy": "median",
        "median": median,
        "q95": q95
    })

p4_report = pd.DataFrame(imputer_stats).sort_values("missing_rate_train", ascending=False)
display(p4_report.head(20))

,col,missing_rate_train,impute_strategy,median,q95
23,same_srv_rate,0.378890,median,1.00,1.0
24,diff_srv_rate,0.376116,median,0.00,0.0
31,dst_host_srv_diff_host_rate,0.312286,median,0.00,0.0
25,srv_diff_host_rate,0.162700,median,0.00,1.0
34,dst_host_rerror_rate,0.095940,median,0.00,1.0
33,dst_host_srv_serror_rate,0.052270,median,0.00,1.0
35,dst_host_srv_rerror_rate,0.049440,median,0.00,1.0
19,serror_rate,0.039599,median,0.00,1.0
21,rerror_rate,0.032143,median,0.00,1.0
20,srv_serror_rate,0.025974,median,0.00,1.0


In [25]:
df_imputed = df.copy()

for c, stats in imputers.items():
    fill_value = stats["median"]
    df_imputed[c] = df_imputed[c].fillna(fill_value)

print("Imputation applied.")

Imputation applied.


In [26]:
# Pastikan numeric tidak ada NaN lagi (kecuali kolom yang sengaja dibiarkan)
post_na = df_imputed[numeric_cols].isna().mean().sort_values(ascending=False)
display(post_na.head(10))

duration             0.0
src_bytes            0.0
dst_bytes            0.0
land                 0.0
wrong_fragment       0.0
urgent               0.0
hot                  0.0
num_failed_logins    0.0
logged_in            0.0
num_compromised      0.0
dtype: float64

In [27]:
df_imputed.to_csv(P4_DIR / "df_p4.csv", index=False)
p4_report.to_csv(P4_DIR / "p4_report.csv", index=False)

with open(P4_DIR / "p4_imputers.json", "w") as f:
    json.dump(imputers, f, indent=2)

print("✅ Saved:", P4_DIR / "df_p4.csv")
print("✅ Saved:", P4_DIR / "p4_report.csv")
print("✅ Saved:", P4_DIR / "p4_imputers.json")

✅ Saved: artifacts_p4/df_p4.csv
✅ Saved: artifacts_p4/p4_report.csv
✅ Saved: artifacts_p4/p4_imputers.json


# P5 - Robust Transform untuk Skew/Outlier/Scale Extremes

In [28]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

P4_DIR = Path("./artifacts_p4")
P0_DIR = Path("./artifacts_p0")
P5_DIR = Path("./artifacts_p5")
P5_DIR.mkdir(exist_ok=True, parents=True)

assert (P4_DIR / "df_p4.csv").exists(), "df_p4.csv tidak ditemukan. Jalankan P4 dulu."
assert (P0_DIR / "train_idx.csv").exists(), "train_idx.csv tidak ditemukan. Jalankan P0 dulu."

df = pd.read_csv(P4_DIR / "df_p4.csv")
train_idx = pd.read_csv(P0_DIR / "train_idx.csv")["idx"].values

# target + flags detection
TARGET_COL = "type_of_attack" if "type_of_attack" in df.columns else None  # fallback
# kalau Anda ingin lebih aman, load dari artifacts:
# (opsional) bisa baca p4_imputers.json untuk target, tapi biasanya target tetap sama
if TARGET_COL is None:
    raise ValueError("Target col tidak terdeteksi. Pastikan kolom target ada (mis. type_of_attack).")

flag_cols = [c for c in df.columns if c.startswith("is_")]
numeric_cols = [c for c in df.columns if c not in flag_cols and c != TARGET_COL and df[c].dtype != "object"]

print("Loaded df_p4:", df.shape)
print("Numeric cols:", len(numeric_cols), "| Flag cols:", len(flag_cols))

Loaded df_p4: (111039, 105)
Numeric cols: 38 | Flag cols: 63


In [29]:
# --- bytes-like detection ---
BYTES_NAME_KEYWORDS = ["bytes", "byte"]  # bisa tambah "src_bytes", "dst_bytes" dll

# --- quantile clip settings (robust) ---
CLIP_LOWER_Q = 0.001
CLIP_UPPER_Q = 0.999

# --- zero-inflation threshold untuk bikin is_nonzero feature ---
ZERO_INFLATION_THRESHOLD = 0.90  # jika >=90% nilai 0 -> buat is_nonzero__col

# --- exclude columns from transform (optional) ---
EXCLUDE_FROM_CLIP = set()  # isi kalau ada kolom yang tidak boleh di-clip

In [30]:
def is_bytes_like(col: str) -> bool:
    c = col.lower()
    return any(k in c for k in BYTES_NAME_KEYWORDS)

bytes_cols = [c for c in numeric_cols if is_bytes_like(c)]
other_num_cols = [c for c in numeric_cols if c not in bytes_cols]

print("Bytes-like cols:", bytes_cols)
print("Other numeric cols:", len(other_num_cols))

Bytes-like cols: ['src_bytes', 'dst_bytes']
Other numeric cols: 36


In [31]:
train_df = df.loc[train_idx]

p5_report_rows = []
clip_bounds = {}      # {col: (lo, hi)}
zero_inflated_cols = []

for c in numeric_cols:
    s = train_df[c]

    # hitung zero-rate (ingat nol = absence, bukan missing)
    zero_rate = float((s == 0).mean())

    # zero-inflation feature
    if zero_rate >= ZERO_INFLATION_THRESHOLD:
        zero_inflated_cols.append(c)

    # quantile bounds untuk clipping (skip jika kolom bytes -> nanti log)
    if c in EXCLUDE_FROM_CLIP:
        lo = hi = None
    else:
        lo = float(s.quantile(CLIP_LOWER_Q))
        hi = float(s.quantile(CLIP_UPPER_Q))

    clip_bounds[c] = {"lo": lo, "hi": hi, "clip_lower_q": CLIP_LOWER_Q, "clip_upper_q": CLIP_UPPER_Q}

    p5_report_rows.append({
        "col": c,
        "is_bytes_like": c in bytes_cols,
        "zero_rate_train": zero_rate,
        "lo_q": lo,
        "hi_q": hi
    })

p5_report = pd.DataFrame(p5_report_rows).sort_values(["is_bytes_like","zero_rate_train"], ascending=[False, False])
display(p5_report.head(30))

print("Zero-inflated cols:", len(zero_inflated_cols))
print("Example:", zero_inflated_cols[:15])

,col,is_bytes_like,zero_rate_train,lo_q,hi_q
2,dst_bytes,True,0.543310,0.0,170545.152
1,src_bytes,True,0.387400,0.0,2194619.000
16,num_outbound_cmds,False,1.000000,0.0,0.000
3,land,False,0.999989,0.0,0.000
5,urgent,False,0.999956,0.0,0.000
14,num_shells,False,0.999689,0.0,0.000
7,num_failed_logins,False,0.999423,0.0,0.000
11,su_attempted,False,0.999323,0.0,0.000
10,root_shell,False,0.998868,0.0,1.000
13,num_file_creations,False,0.997770,0.0,2.000


Zero-inflated cols: 19
Example: ['duration', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'rerror_rate']


In [32]:
df_p5 = df.copy()

# (A) Activation features: is_nonzero__col
nonzero_feature_cols = []
for c in zero_inflated_cols:
    feat = f"is_nonzero__{c}"
    df_p5[feat] = (df_p5[c] != 0).astype("int8")
    nonzero_feature_cols.append(feat)

print("Added is_nonzero features:", len(nonzero_feature_cols))

# (B) Clip per feature (gunakan bounds dari train)
for c in numeric_cols:
    b = clip_bounds.get(c, None)
    if b is None or b["lo"] is None or b["hi"] is None:
        continue
    lo, hi = b["lo"], b["hi"]
    df_p5[c] = df_p5[c].clip(lower=lo, upper=hi)

# (C) log1p untuk bytes cols (nol tetap nol)
log1p_cols = []
for c in bytes_cols:
    # log1p aman untuk >=0; kalau ada negatif (harusnya tidak), kita guard:
    if (df_p5[c] < 0).any():
        # shift minimal (harusnya jarang)
        minv = float(df_p5[c].min())
        df_p5[c] = np.log1p(df_p5[c] - minv)
    else:
        df_p5[c] = np.log1p(df_p5[c])
    log1p_cols.append(c)

print("Applied log1p to bytes cols:", log1p_cols)

Added is_nonzero features: 19
Applied log1p to bytes cols: ['src_bytes', 'dst_bytes']


In [33]:
# cek apakah ada inf/NaN baru pada numeric
num_check = [c for c in numeric_cols if c in df_p5.columns]
inf_count = int(np.isinf(df_p5[num_check]).sum().sum())
nan_count = int(df_p5[num_check].isna().sum().sum())

print("Inf count (numeric):", inf_count)
print("NaN count (numeric):", nan_count)

# quick view distribution shift for a few cols
display(p5_report.sort_values("zero_rate_train", ascending=False).head(10))

Inf count (numeric): 0
NaN count (numeric): 0


,col,is_bytes_like,zero_rate_train,lo_q,hi_q
16,num_outbound_cmds,False,1.000000,0.0,0.0
3,land,False,0.999989,0.0,0.0
5,urgent,False,0.999956,0.0,0.0
14,num_shells,False,0.999689,0.0,0.0
7,num_failed_logins,False,0.999423,0.0,0.0
11,su_attempted,False,0.999323,0.0,0.0
10,root_shell,False,0.998868,0.0,1.0
13,num_file_creations,False,0.997770,0.0,2.0
15,num_access_files,False,0.997115,0.0,1.0
12,num_root,False,0.996150,0.0,9.0


In [34]:
df_p5.to_csv(P5_DIR / "df_p5.csv", index=False)
p5_report.to_csv(P5_DIR / "p5_report.csv", index=False)

p5_transform = {
    "stage": "P5",
    "target_col": TARGET_COL,
    "clip_lower_q": CLIP_LOWER_Q,
    "clip_upper_q": CLIP_UPPER_Q,
    "zero_inflation_threshold": ZERO_INFLATION_THRESHOLD,
    "bytes_cols_log1p": log1p_cols,
    "zero_inflated_cols": zero_inflated_cols,
    "nonzero_feature_cols": nonzero_feature_cols,
    "clip_bounds": clip_bounds,   # per-col bounds fitted on train
}

with open(P5_DIR / "p5_transform.json", "w") as f:
    json.dump(p5_transform, f, indent=2)

print("✅ Saved:", P5_DIR / "df_p5.csv")
print("✅ Saved:", P5_DIR / "p5_report.csv")
print("✅ Saved:", P5_DIR / "p5_transform.json")

✅ Saved: artifacts_p5/df_p5.csv
✅ Saved: artifacts_p5/p5_report.csv
✅ Saved: artifacts_p5/p5_transform.json


# P6 - Encoding Kategorikal (Controlled, anti-explosion)

In [35]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

P5_DIR = Path("./artifacts_p5")
P0_DIR = Path("./artifacts_p0")
P6_DIR = Path("./artifacts_p6")
P6_DIR.mkdir(exist_ok=True, parents=True)

assert (P5_DIR / "df_p5.csv").exists(), "df_p5.csv tidak ditemukan. Jalankan P5 dulu."
assert (P0_DIR / "train_idx.csv").exists(), "train_idx.csv tidak ditemukan. Jalankan P0 dulu."
assert (P0_DIR / "val_idx.csv").exists(), "val_idx.csv tidak ditemukan. Jalankan P0 dulu."

df = pd.read_csv(P5_DIR / "df_p5.csv")
train_idx = pd.read_csv(P0_DIR / "train_idx.csv")["idx"].values
val_idx = pd.read_csv(P0_DIR / "val_idx.csv")["idx"].values

# target col (sesuaikan jika bukan type_of_attack)
TARGET_COL = "type_of_attack"
assert TARGET_COL in df.columns, f"Target '{TARGET_COL}' tidak ada."

print("Loaded df_p5:", df.shape)
print("Train:", len(train_idx), "| Val:", len(val_idx))

Loaded df_p5: (111039, 124)
Train: 90129 | Val: 20910


In [36]:
# Controlled categorical columns (edit jika beda nama)
CAT_ONEHOT_COLS = ["protocol_type", "flag"]
SERVICE_COL = "service"

# cek kolom tersedia
available = set(df.columns)
CAT_ONEHOT_COLS = [c for c in CAT_ONEHOT_COLS if c in available]
use_service = SERVICE_COL in available

print("CAT_ONEHOT_COLS:", CAT_ONEHOT_COLS)
print("Use service:", use_service)

CAT_ONEHOT_COLS: ['protocol_type', 'flag']
Use service: True


In [37]:
RARE_MIN_FREQ = 50          # minimal count di train agar tidak dianggap rare
RARE_MIN_RATE = 0.001       # atau minimal proporsi (0.1%)
OTHER_TOKEN = "other"

train_df = df.loc[train_idx]

service_keep = None
if use_service:
    svc = train_df[SERVICE_COL].astype("string").fillna(OTHER_TOKEN).str.strip().str.lower()
    vc = svc.value_counts(dropna=False)

    n_train = len(train_df)
    keep_by_count = set(vc[vc >= RARE_MIN_FREQ].index)
    keep_by_rate = set(vc[(vc / n_train) >= RARE_MIN_RATE].index)

    service_keep = sorted(list(keep_by_count.union(keep_by_rate)))
    print("Service categories kept:", len(service_keep))
    print("Top service:\n", vc.head(10))

def normalize_cat(s: pd.Series) -> pd.Series:
    return s.astype("string").fillna(OTHER_TOKEN).str.strip().str.lower()

def apply_other_mapping(s: pd.Series, keep_list: list[str] | None):
    s = normalize_cat(s)
    if keep_list is None:
        return s
    return s.where(s.isin(keep_list), OTHER_TOKEN)

Service categories kept: 60
Top service:
 service
http        28629
private     15544
domain_u     6388
smtp         5298
ftp_data     4231
eco_i        3787
other        3678
ecr_i        2405
telnet       1680
finger       1279
Name: count, dtype: Int64


In [38]:
# 1) siapkan kategori train reference untuk protocol/flag
cat_levels = {}
for c in CAT_ONEHOT_COLS:
    levels = sorted(normalize_cat(train_df[c]).unique().tolist())
    cat_levels[c] = levels
    print(f"{c} levels(train):", levels)

# 2) buat copy data kategori yang sudah dinormalisasi + mapped
df_cat = pd.DataFrame(index=df.index)

for c in CAT_ONEHOT_COLS:
    s = normalize_cat(df[c])
    # unknown -> other
    s = s.where(s.isin(cat_levels[c]), OTHER_TOKEN)
    df_cat[c] = s

if use_service:
    df_cat[SERVICE_COL] = apply_other_mapping(df[SERVICE_COL], service_keep)

# 3) one-hot encode (full df) lalu align ke train columns template
df_dum = pd.get_dummies(df_cat, columns=df_cat.columns, dummy_na=False)

# template columns from train only
train_dum = pd.get_dummies(df_cat.loc[train_idx], columns=df_cat.columns, dummy_na=False)
template_cols = train_dum.columns.tolist()

# align full to template: drop extras, add missing
df_dum = df_dum.reindex(columns=template_cols, fill_value=0).astype("int8")

print("Dummy feature shape:", df_dum.shape)
print("Example dummy cols:", df_dum.columns[:20].tolist())

protocol_type levels(train): ['icmp', 'other', 'tcp', 'udp']
flag levels(train): ['oth', 'other', 'rej', 'rsto', 'rstos0', 'rstr', 's0', 's1', 's2', 's3', 'sf', 'sh']
Dummy feature shape: (111039, 76)
Example dummy cols: ['protocol_type_icmp', 'protocol_type_other', 'protocol_type_tcp', 'protocol_type_udp', 'flag_oth', 'flag_other', 'flag_rej', 'flag_rsto', 'flag_rstos0', 'flag_rstr', 'flag_s0', 'flag_s1', 'flag_s2', 'flag_s3', 'flag_sf', 'flag_sh', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns']


In [39]:
# =========================
# P6 — Cell 5 (PATCH)
# =========================

# Numeric + flags columns (exclude target and object categoricals)
flag_cols = [c for c in df.columns if c.startswith("is_")]
num_cols = [c for c in df.columns if c not in flag_cols and c != TARGET_COL and df[c].dtype != "object"]

X_num = df[num_cols].copy()
X_flag = df[flag_cols].copy()

# pastikan dtype ringkas
X_num = X_num.astype("float32")

# FIX: bersihkan NA/inf sebelum cast ke int8
X_flag = X_flag.replace([np.inf, -np.inf], np.nan).fillna(0)

# (opsional) pastikan flag benar-benar integer kecil (0/1). kalau ada yang >1 (mis row_anomaly_score),
# kita tetap izinkan, tapi amankan ke rentang int8
X_flag = X_flag.clip(lower=-128, upper=127)

# cast
X_flag = X_flag.astype("int8")

# final X
X = pd.concat([X_num, X_flag, df_dum], axis=1)
y = df[TARGET_COL].astype("string")

print("X shape:", X.shape, "| y shape:", y.shape)
print("X dtypes summary:\n", X.dtypes.value_counts())
print("Any NaN in X?", bool(X.isna().any().any()))

X shape: (111039, 196) | y shape: (111039,)
X dtypes summary:
 int8       158
float32     38
Name: count, dtype: int64
Any NaN in X? False


In [40]:
# Save feature names
feature_names = X.columns.tolist()
with open(P6_DIR / "feature_names_p6.json", "w") as f:
    json.dump(feature_names, f, indent=2)

# Save encoder artifacts
p6_artifacts = {
    "stage": "P6",
    "target_col": TARGET_COL,
    "cat_onehot_cols": CAT_ONEHOT_COLS,
    "service_col": SERVICE_COL if use_service else None,
    "other_token": OTHER_TOKEN,
    "rare_min_freq": RARE_MIN_FREQ,
    "rare_min_rate": RARE_MIN_RATE,
    "cat_levels_train": {k: v for k, v in cat_levels.items()},
    "service_keep": service_keep if use_service else None,
    "dummy_template_cols": template_cols,
    "n_features_total": int(len(feature_names)),
    "n_features_dummy": int(df_dum.shape[1]),
    "n_features_numeric": int(X_num.shape[1]),
    "n_features_flag": int(X_flag.shape[1]),
}

with open(P6_DIR / "p6_encoder_artifacts.json", "w") as f:
    json.dump(p6_artifacts, f, indent=2)

# Save full X,y (parquet + csv for y)
X.to_parquet(P6_DIR / "X_p6.parquet", index=False)
y.to_frame("y").to_csv(P6_DIR / "y.csv", index=False)

# Save split datasets (train/val) optional (parquet)
X.loc[train_idx].to_parquet(P6_DIR / "X_train.parquet", index=False)
X.loc[val_idx].to_parquet(P6_DIR / "X_val.parquet", index=False)
y.loc[train_idx].to_frame("y").to_csv(P6_DIR / "y_train.csv", index=False)
y.loc[val_idx].to_frame("y").to_csv(P6_DIR / "y_val.csv", index=False)

print("✅ Saved:", P6_DIR / "X_p6.parquet")
print("✅ Saved:", P6_DIR / "X_train.parquet")
print("✅ Saved:", P6_DIR / "X_val.parquet")
print("✅ Saved:", P6_DIR / "p6_encoder_artifacts.json")
print("✅ Saved:", P6_DIR / "feature_names_p6.json")

✅ Saved: artifacts_p6/X_p6.parquet
✅ Saved: artifacts_p6/X_train.parquet
✅ Saved: artifacts_p6/X_val.parquet
✅ Saved: artifacts_p6/p6_encoder_artifacts.json
✅ Saved: artifacts_p6/feature_names_p6.json


In [41]:
# Near-constant dummy features (top ratio)
train_X = X.loc[train_idx]
top_ratio = (train_X.mean(axis=0)).clip(0,1)  # for int8 dummies it works
near_const = top_ratio[(top_ratio >= 0.995) | (top_ratio <= 0.005)].sort_values(ascending=False)

print("Near-constant features count:", len(near_const))
display(near_const.head(20))

Near-constant features count: 116


duration                                   1.000000
src_bytes                                  1.000000
dst_bytes                                  1.000000
dst_host_srv_count                         1.000000
srv_count                                  1.000000
dst_host_count                             1.000000
row_anomaly_score                          1.000000
__group_id__                               1.000000
count                                      1.000000
is_missing_implisit__dst_host_srv_count    0.004893
is_missing_implisit__duration              0.004871
service_vmnet                              0.004849
service_imap4                              0.004838
is_sentinel_99999__rerror_rate             0.004804
service_csnet_ns                           0.004738
is_missing_implisit__wrong_fragment        0.004505
service_domain                             0.004438
service_supdup                             0.004416
service_discard                            0.004383
service_ctf 

# P7 - Anti-Shortcut Guardrails

In [42]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

P6_DIR = Path("./artifacts_p6")
P7_DIR = Path("./artifacts_p7")
P7_DIR.mkdir(exist_ok=True, parents=True)

assert (P6_DIR / "X_train.parquet").exists(), "X_train.parquet tidak ditemukan. Pastikan P6 sukses."
assert (P6_DIR / "X_val.parquet").exists(), "X_val.parquet tidak ditemukan."
assert (P6_DIR / "y_train.csv").exists(), "y_train.csv tidak ditemukan."
assert (P6_DIR / "y_val.csv").exists(), "y_val.csv tidak ditemukan."

X_train = pd.read_parquet(P6_DIR / "X_train.parquet")
X_val   = pd.read_parquet(P6_DIR / "X_val.parquet")
y_train = pd.read_csv(P6_DIR / "y_train.csv")["y"].astype(str)
y_val   = pd.read_csv(P6_DIR / "y_val.csv")["y"].astype(str)

with open(P6_DIR / "p6_encoder_artifacts.json", "r") as f:
    p6 = json.load(f)

print("X_train:", X_train.shape, "| X_val:", X_val.shape)
print("y_train classes:", y_train.nunique(), "| y_val classes:", y_val.nunique())

X_train: (90129, 196) | X_val: (20910, 196)
y_train classes: 8 | y_val classes: 8


In [43]:
feature_names = X_train.columns.tolist()

# dummy columns are produced from get_dummies; in our pipeline they are exactly template cols
dummy_cols = p6.get("dummy_template_cols", [])
dummy_cols = [c for c in dummy_cols if c in feature_names]

# flag columns start with "is_"
flag_cols = [c for c in feature_names if c.startswith("is_")]

# dirty sentinel flags
sentinel_flag_cols = [c for c in flag_cols if c.startswith("is_sentinel_99999__")]
missing_flag_cols  = [c for c in flag_cols if c.startswith("is_missing_implisit__")]
strnan_flag_cols   = [c for c in flag_cols if c.startswith("is_str_nan__")]

# row anomaly score (kalau ada)
row_anom_col = "row_anomaly_score" if "row_anomaly_score" in feature_names else None

print("dummy_cols:", len(dummy_cols))
print("flag_cols:", len(flag_cols))
print("sentinel_flag_cols:", len(sentinel_flag_cols))
print("missing_flag_cols:", len(missing_flag_cols))
print("strnan_flag_cols:", len(strnan_flag_cols))
print("row_anomaly_score exists:", row_anom_col is not None)

dummy_cols: 76
flag_cols: 82
sentinel_flag_cols: 28
missing_flag_cols: 33
strnan_flag_cols: 0
row_anomaly_score exists: True


In [44]:
def drop_cols(base_cols, cols_to_drop):
    drop_set = set(cols_to_drop)
    return [c for c in base_cols if c not in drop_set]

ALL_COLS = feature_names

scenarios = {
    "FULL": ALL_COLS,

    "DROP_SENTINEL_FLAGS": drop_cols(ALL_COLS, sentinel_flag_cols),

    "DROP_MISSING_FLAGS": drop_cols(
        ALL_COLS,
        missing_flag_cols + ( [row_anom_col] if row_anom_col else [] )
    ),

    "DROP_CAT_DUMMIES": drop_cols(ALL_COLS, dummy_cols),

    "DROP_DIRTY_ALL": drop_cols(
        ALL_COLS,
        sentinel_flag_cols + missing_flag_cols + strnan_flag_cols + dummy_cols + ( [row_anom_col] if row_anom_col else [] )
    ),
}

for k,v in scenarios.items():
    print(k, "n_features:", len(v))

FULL n_features: 196
DROP_SENTINEL_FLAGS n_features: 168
DROP_MISSING_FLAGS n_features: 162
DROP_CAT_DUMMIES n_features: 120
DROP_DIRTY_ALL n_features: 58


In [45]:
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier

def evaluate_model(Xtr, ytr, Xva, yva, model_name="hgb"):
    if model_name == "hgb":
        model = HistGradientBoostingClassifier(
            learning_rate=0.1,
            max_depth=8,
            max_iter=300,
            random_state=42
        )
    elif model_name == "logreg":
        model = LogisticRegression(
            max_iter=2000,
            n_jobs=-1,
            class_weight="balanced"  # penting untuk long-tail
        )
    else:
        raise ValueError("unknown model_name")

    model.fit(Xtr, ytr)
    pred = model.predict(Xva)

    macro_f1 = f1_score(yva, pred, average="macro")
    weighted_f1 = f1_score(yva, pred, average="weighted")

    # per-class recall from report dict
    rep = classification_report(yva, pred, output_dict=True, zero_division=0)
    per_class_recall = {k: v["recall"] for k,v in rep.items() if k not in ["accuracy","macro avg","weighted avg"]}

    return macro_f1, weighted_f1, per_class_recall

# quick sanity baseline run

In [46]:
results_rows = []
perclass_store = {}

for scen, cols in scenarios.items():
    Xtr = X_train[cols]
    Xva = X_val[cols]

    # Model 1: HGB (non-linear, good for interactions)
    m1_macro, m1_weighted, m1_recall = evaluate_model(Xtr, y_train, Xva, y_val, model_name="hgb")

    # Model 2: LogReg (linear baseline; if it wins too hard → suspicion)
    m2_macro, m2_weighted, m2_recall = evaluate_model(Xtr, y_train, Xva, y_val, model_name="logreg")

    results_rows.append({
        "scenario": scen,
        "n_features": len(cols),
        "HGB_macroF1": m1_macro,
        "HGB_weightedF1": m1_weighted,
        "LOGREG_macroF1": m2_macro,
        "LOGREG_weightedF1": m2_weighted,
    })

    perclass_store[(scen, "HGB")] = m1_recall
    perclass_store[(scen, "LOGREG")] = m2_recall

results = pd.DataFrame(results_rows).sort_values("HGB_macroF1", ascending=False)
display(results)

,scenario,n_features,HGB_macroF1,HGB_weightedF1,LOGREG_macroF1,LOGREG_weightedF1
1,DROP_SENTINEL_FLAGS,168,0.963622,0.995742,0.003234,0.000339
2,DROP_MISSING_FLAGS,162,0.962165,0.995373,0.003234,0.000339
4,DROP_DIRTY_ALL,58,0.961183,0.995690,0.003234,0.000339
0,FULL,196,0.954346,0.994877,0.003234,0.000339
3,DROP_CAT_DUMMIES,120,0.893538,0.989028,0.003234,0.000339


In [47]:
def add_deltas(df, base="FULL", col="HGB_macroF1"):
    base_val = float(df.loc[df["scenario"] == base, col].values[0])
    df[f"delta_{col}_vs_{base}"] = df[col] - base_val
    return df

results2 = results.copy()
results2 = add_deltas(results2, base="FULL", col="HGB_macroF1")
results2 = add_deltas(results2, base="FULL", col="LOGREG_macroF1")
display(results2.sort_values("scenario"))

,scenario,n_features,HGB_macroF1,HGB_weightedF1,LOGREG_macroF1,LOGREG_weightedF1,delta_HGB_macroF1_vs_FULL,delta_LOGREG_macroF1_vs_FULL
3,DROP_CAT_DUMMIES,120,0.893538,0.989028,0.003234,0.000339,-0.060807,0.0
4,DROP_DIRTY_ALL,58,0.961183,0.995690,0.003234,0.000339,0.006838,0.0
2,DROP_MISSING_FLAGS,162,0.962165,0.995373,0.003234,0.000339,0.007819,0.0
1,DROP_SENTINEL_FLAGS,168,0.963622,0.995742,0.003234,0.000339,0.009276,0.0
0,FULL,196,0.954346,0.994877,0.003234,0.000339,0.000000,0.0


In [48]:
# Ambil list kelas dari y_val
classes = sorted(y_val.unique().tolist())

def recall_table_for(scen, model_key="HGB"):
    rec = perclass_store[(scen, model_key)]
    return pd.DataFrame({"class": classes, "recall": [rec.get(c, 0.0) for c in classes]}).sort_values("recall")

# Bandingkan FULL vs DROP_DIRTY_ALL (HGB)
full_rec = recall_table_for("FULL", "HGB").rename(columns={"recall": "recall_FULL"})
drop_rec = recall_table_for("DROP_DIRTY_ALL", "HGB").rename(columns={"recall": "recall_DROP_DIRTY_ALL"})

comp = full_rec.merge(drop_rec, on="class", how="outer").fillna(0)
comp["delta_recall"] = comp["recall_DROP_DIRTY_ALL"] - comp["recall_FULL"]

display(comp.sort_values("delta_recall").head(20))   # kelas yang paling jatuh
display(comp.sort_values("delta_recall", ascending=False).head(20))  # kelas yang membaik

,class,recall_FULL,recall_DROP_DIRTY_ALL,delta_recall
1,ipsweep,0.974026,0.913420,-0.060606
5,portsweep,0.994764,0.989529,-0.005236
0,Denial of Service Attack,0.962441,0.957746,-0.004695
2,neptune,0.999859,0.999859,0.000000
7,smurf,0.996350,0.996350,0.000000
4,normal,0.998987,0.998987,0.000000
6,satan,0.986111,0.992063,0.005952
3,nmap,0.648485,0.818182,0.169697


,class,recall_FULL,recall_DROP_DIRTY_ALL,delta_recall
3,nmap,0.648485,0.818182,0.169697
6,satan,0.986111,0.992063,0.005952
4,normal,0.998987,0.998987,0.000000
2,neptune,0.999859,0.999859,0.000000
7,smurf,0.996350,0.996350,0.000000
0,Denial of Service Attack,0.962441,0.957746,-0.004695
5,portsweep,0.994764,0.989529,-0.005236
1,ipsweep,0.974026,0.913420,-0.060606


In [49]:
results2.to_csv(P7_DIR / "p7_ablation_results.csv", index=False)

# Save per-class recall (json)
with open(P7_DIR / "p7_perclass_recall.json", "w") as f:
    json.dump({f"{k[0]}__{k[1]}": v for k,v in perclass_store.items()}, f, indent=2)

# Simple heuristic gate decision
# (Anda bisa ubah rule sesuai preferensi)
full_hgb = float(results2.loc[results2["scenario"]=="FULL", "HGB_macroF1"].values[0])
dirty_hgb = float(results2.loc[results2["scenario"]=="DROP_DIRTY_ALL", "HGB_macroF1"].values[0])
drop = dirty_hgb - full_hgb

decision = "✅ Go" if drop >= -0.05 else ("⚠️ Go with caution" if drop >= -0.12 else "❌ Stop & re-scope")

gate = {
    "full_hgb_macroF1": full_hgb,
    "drop_dirty_all_hgb_macroF1": dirty_hgb,
    "delta_drop_dirty_all": drop,
    "decision": decision,
    "notes": [
        "Jika drop besar: indikasi model mengandalkan dirty/sentinel/categorical shortcuts.",
        "Perhatikan kelas minoritas yang recall-nya jatuh saat DROP_DIRTY_ALL."
    ]
}

with open(P7_DIR / "p7_gate.json", "w") as f:
    json.dump(gate, f, indent=2)

print("✅ Saved:", P7_DIR / "p7_ablation_results.csv")
print("✅ Saved:", P7_DIR / "p7_perclass_recall.json")
print("✅ Saved:", P7_DIR / "p7_gate.json")
print("Gate decision:", decision)
print("Delta DROP_DIRTY_ALL vs FULL (HGB macroF1):", drop)

✅ Saved: artifacts_p7/p7_ablation_results.csv
✅ Saved: artifacts_p7/p7_perclass_recall.json
✅ Saved: artifacts_p7/p7_gate.json
Gate decision: ✅ Go
Delta DROP_DIRTY_ALL vs FULL (HGB macroF1): 0.006837975541045815


# P8 - Spliting

In [50]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
import hashlib

P6_DIR = Path("./artifacts_p6")
FINAL_DIR = Path("./artifacts_final")
FINAL_DIR.mkdir(exist_ok=True, parents=True)

X_train = pd.read_parquet(P6_DIR / "X_train.parquet")
X_val   = pd.read_parquet(P6_DIR / "X_val.parquet")
y_train = pd.read_csv(P6_DIR / "y_train.csv")["y"].astype(str)
y_val   = pd.read_csv(P6_DIR / "y_val.csv")["y"].astype(str)

with open(P6_DIR / "feature_names_p6.json", "r") as f:
    feature_names = json.load(f)

print("X_train:", X_train.shape, "X_val:", X_val.shape)
print("n_features:", len(feature_names))

X_train: (90129, 196) X_val: (20910, 196)
n_features: 196


In [51]:
# Fit label encoder on train only (but keep all labels observed)
classes = sorted(y_train.unique().tolist())
label2id = {c:i for i,c in enumerate(classes)}
id2label = {i:c for c,i in label2id.items()}

y_train_enc = y_train.map(label2id).astype("int32").to_numpy()
y_val_enc   = y_val.map(label2id).astype("int32").to_numpy()

assert not np.isnan(y_train_enc).any()
assert not np.isnan(y_val_enc).any()

label_encoder = {"classes": classes, "label2id": label2id, "id2label": id2label}
print("n_classes:", len(classes))
print("Example mapping:", list(label2id.items())[:10])

n_classes: 8
Example mapping: [('Denial of Service Attack', 0), ('ipsweep', 1), ('neptune', 2), ('nmap', 3), ('normal', 4), ('portsweep', 5), ('satan', 6), ('smurf', 7)]


In [52]:
# Save X
X_train.to_parquet(FINAL_DIR / "X_train.parquet", index=False)
X_val.to_parquet(FINAL_DIR / "X_val.parquet", index=False)

# Save y
np.save(FINAL_DIR / "y_train.npy", y_train_enc)
np.save(FINAL_DIR / "y_val.npy", y_val_enc)

# Save metadata
with open(FINAL_DIR / "feature_names.json", "w") as f:
    json.dump(feature_names, f, indent=2)

with open(FINAL_DIR / "label_encoder.json", "w") as f:
    json.dump(label_encoder, f, indent=2)

print("✅ Saved training bundle to:", FINAL_DIR)
print(list(FINAL_DIR.glob("*")))

✅ Saved training bundle to: artifacts_final
[PosixPath('artifacts_final/y_val.npy'), PosixPath('artifacts_final/X_val.parquet'), PosixPath('artifacts_final/feature_names.json'), PosixPath('artifacts_final/label_encoder.json'), PosixPath('artifacts_final/y_train.npy'), PosixPath('artifacts_final/X_train.parquet')]


In [53]:
def file_md5(path: Path, nbytes=2_000_000):
    # hash ringan: hanya first nbytes
    h = hashlib.md5()
    with open(path, "rb") as f:
        h.update(f.read(nbytes))
    return h.hexdigest()

manifest = {
    "bundle_dir": str(FINAL_DIR),
    "X_train_path": str(FINAL_DIR / "X_train.parquet"),
    "X_val_path": str(FINAL_DIR / "X_val.parquet"),
    "y_train_path": str(FINAL_DIR / "y_train.npy"),
    "y_val_path": str(FINAL_DIR / "y_val.npy"),
    "feature_names_path": str(FINAL_DIR / "feature_names.json"),
    "label_encoder_path": str(FINAL_DIR / "label_encoder.json"),
    "shapes": {
        "X_train": list(X_train.shape),
        "X_val": list(X_val.shape),
        "y_train": [int(y_train_enc.shape[0])],
        "y_val": [int(y_val_enc.shape[0])]
    },
    "hashes_md5_head": {
        "X_train": file_md5(FINAL_DIR / "X_train.parquet"),
        "X_val": file_md5(FINAL_DIR / "X_val.parquet"),
        "y_train": file_md5(FINAL_DIR / "y_train.npy"),
        "y_val": file_md5(FINAL_DIR / "y_val.npy"),
    },
    "notes": [
        "Preprocessing pipeline stages: P0..P6",
        "This bundle is ready for modeling notebooks."
    ]
}

with open(FINAL_DIR / "dataset_manifest.json", "w") as f:
    json.dump(manifest, f, indent=2)

print("✅ Saved:", FINAL_DIR / "dataset_manifest.json")

✅ Saved: artifacts_final/dataset_manifest.json
